In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
import quantecon as qe

## Question 2: Solve for representative agent steady state

In [12]:
qe?

In [10]:
def params1():
    
    θ = 0.21
    ν = 0.64
    δ = 0.1
    β = 0.96
    ϕ = 0.5
    ρ = 0.9
    σ = 0.02
    cf = 0.01
    ce = 0.1
    
    params = θ, ν, δ, β, ϕ, ρ, σ, cf, ce
    
    return params

In [21]:
def get_markov_shocks(*params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    # markov approximation of shocks using quantecon
    shock_matrix = qe.markov.approximation.tauchen(ρ, σ, n = 10)
    
    return shock_matrix

def get_SS_shocks(markov_shocks):
    
    SS_shock = markov_shocks.stationary_distributions
    
    return SS_shock

def get_w(K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    w = θ * np.exp(shock) * K**(θ-1) * N**ν
    
    return w

def get_r(K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    r = ν * np.exp(shock) * K**θ * N**(ν-1)
    
    return r

def get_c(w, r, K, N, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    c = w * N + r * K - (K - (1 - δ) * K)
    
    return c

def SS_rep(K, N, shock, *params):
    
    # unpack params
    θ, ν, δ, β, ϕ, ρ, σ, cf, ce = params
    
    w = get_w(K, N, shock, *params)
    r = get_r(K, N, shock, *params)
    c = get_c(w, r, K, N, *params)
    
    f1 = β * (1/c) * (1 - δ + r) - (1/c)
    
    return f1

opt.fsolve(SS_rep, 0.5, args = (0.6, get_mean_TFP(get_markov_shocks(*params1())), *params1()))

array([2.09597199e+83])